<a href="https://colab.research.google.com/github/OhJin-Soo/ktb_ai/blob/main/pycuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 12.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=4b27c40c81331b2bb46b98ad05c18ed88125263e3ae9b582387102eb7e65384b
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


In [8]:
import pycuda.compiler as comp
import pycuda.driver as drv
import numpy
import pycuda.autoinit

#pycuda.compiler: CUDA C 코드를 컴파일하기 위한 모듈

#pycuda.driver: GPU 디바이스와 상호작용하기 위한 모듈

#numpy: CPU에서 데이터 생성 및 처리를 위한 라이브러리

#pycuda.autoinit: CUDA 컨텍스트를 자동으로 초기화 (GPU 사용 준비)

mod = comp.SourceModule(
    """
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
"""
)

#SourceModule에 문자열로 CUDA C 코드를 작성하면, PyCUDA가 이를 컴파일합니다.

#__global__ void multiply_them(...)

#__global__은 GPU에서 실행되는 커널 함수임을 의미합니다.

#threadIdx.x
#현재 스레드의 인덱스.
#CUDA는 수많은 스레드가 병렬로 실행되므로, 각 스레드가 처리할 데이터를 구분하기 위해 사용됩니다.

#dest[i] = a[i] * b[i]
#각 스레드가 자신의 인덱스 i에 해당하는 요소를 곱해서 결과를 dest에 저장합니다.

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(400).astype(numpy.float32)
b = numpy.random.randn(400).astype(numpy.float32)

#a와 b는 CPU에서 생성한 난수 벡터 (float32)
#dest는 결과를 담을 빈 배열
#참고: GPU 연산에서는 데이터 타입이 float32여야 호환이 잘 됩니다.

dest = numpy.zeros_like(a)
multiply_them(drv.Out(dest), drv.In(a), drv.In(b), block=(400, 1, 1))

#drv.In(a): CPU 배열을 GPU로 복사
#drv.Out(dest): GPU 연산 결과를 다시 CPU 배열로 복사
#block=(400, 1, 1):
#스레드 블록 크기를 설정.
#여기서는 400개의 스레드를 1차원으로 배치
#즉, 각 스레드가 1개의 요소를 처리
#주의: 이 예제에서는 400개의 스레드를 한 블록에 넣었지만, 실제 큰 배열에서는 여러 블록으로 나눠야 합니다.

print(dest - a * b)

#GPU 연산 결과 dest와 CPU에서 직접 계산한 a*b를 비교
#거의 0에 가까운 값이 나와야 정상 (부동소수점 오차 수준)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [12]:
import cupy

#CuPy는 NumPy와 거의 동일한 API를 제공하지만, 내부 연산을 CUDA GPU에서 실행합니다.
#즉, numpy 대신 cupy를 쓰면 자동으로 GPU 가속이 되는 느낌.

a = cupy.random.randn(400)
b = cupy.random.randn(400)

#cupy.random.randn → GPU 메모리 위에서 난수 배열 생성 (numpy.random.randn과 같은 역할, but GPU 버전).
#결과는 GPU 메모리에 저장된 배열(cupy.ndarray).

dest = cupy.zeros_like(a)

#a와 같은 shape, dtype을 가지는 GPU 배열 생성 (모두 0으로 초기화).

print(dest - a * b)

#a * b: GPU에서 벡터 곱 (원소별 곱).
#dest - a * b: 결과를 빼기 → 사실 dest는 0이므로, 결과는 단순히 -a*b.
#즉, "a와 b의 원소별 곱을 구한 뒤, 그것에 마이너스를 붙여서 출력" 한 것.

[-9.11687716e-02 -2.82092405e-01  1.56302978e-01  8.99207677e-01
  3.14157732e-01  1.31768510e+00 -4.72836709e-02 -1.14595615e-01
 -2.55442244e-01  3.09299746e+00 -3.29755154e-01 -6.08423389e-01
 -3.31611400e-01  5.44487054e-01 -5.16687183e-01 -7.72562122e-01
  8.99675581e-01  1.16866174e-01  1.59935167e-01 -3.63125248e-01
 -1.43849025e-01  6.08904930e-01 -7.54232787e-02 -4.13442076e-01
 -2.73772939e-01  5.87625624e-01 -3.09636121e+00 -8.64615811e-02
  1.06907107e+00 -1.05752071e+00 -2.54348813e-01  2.38669069e-01
  4.13698282e-02  3.33077784e-01 -4.39438685e-02 -3.57990095e+00
  6.51075337e-01  1.87956050e-01  6.15157298e-02  4.51325003e-01
  4.15904762e-02 -3.11410153e-01 -5.80417879e-02 -2.11261702e-02
 -1.44597404e+00 -9.70216198e-01  2.15212350e-01  3.22572894e-02
  1.94744761e+00 -1.43345498e+00  1.51876221e-01  4.32512857e-01
  1.70544603e-01 -1.46244405e+00 -2.10011680e-01 -5.78880929e-01
 -1.87465729e-01  1.38300144e+00 -3.31405061e-02 -1.08517473e-01
  3.47502943e-01  5.23031